In [1]:
import os
import services as _services



In [15]:
files_name = list(map(lambda x: os.path.basename(x).replace(".csv",""),os.listdir("data")))
files_name

['departments_t', 'hired_employees_t', 'jobs_t']

In [4]:
file_paths = [f"data/{filename}.csv" for filename in _services.get_all_files("data")]
print(file_paths)


['data/departments_t.csv', 'data/hired_employees_t.csv', 'data/jobs_t.csv']
